In [48]:
using SparsityDetection, SparseArrays
using SparseDiffTools
using FiniteDiff
import LinearAlgebra as la
import ForwardDiff

In [3]:
fcalls = 0
function f(y,x) # in-place
  global fcalls += 1
  for i in 2:length(x)-1
    y[i] = x[i-1] - 2x[i] + x[i+1]
  end
  y[1] = -2x[1] + x[2]
  y[end] = x[end-1] - 2x[end]
  nothing
end

function g(x) # out-of-place
  global fcalls += 1
  y = zero(x)
  for i in 2:length(x)-1
    y[i] = x[i-1] - 2x[i] + x[i+1]
  end
  y[1] = -2x[1] + x[2]
  y[end] = x[end-1] - 2x[end]
  y
end

g (generic function with 1 method)

In [40]:
input = rand(30)
output = similar(input)
sparsity_pattern = jacobian_sparsity(f,output,input)
jac = Float64.(sparse(sparsity_pattern))

Explored path: SparsityDetection.Path(Bool[], 1)


30×30 SparseMatrixCSC{Float64, Int64} with 88 stored entries:
⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛

In [41]:
colors = matrix_colors(jac);

In [42]:
@time ForwardDiff.jacobian(g, input);

  0.000019 seconds (7 allocations: 19.781 KiB)


In [45]:
@time forwarddiff_color_jacobian!(jac, f, input, colorvec = colors)

  0.000083 seconds (239 allocations: 13.203 KiB)


30×30 SparseMatrixCSC{Float64, Int64} with 88 stored entries:
⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛

In [57]:
@time FiniteDiff.finite_difference_jacobian(g, rand(30), colorvec=colors)

  0.000026 seconds (44 allocations: 7.469 KiB)


30×3 Matrix{Float64}:
 -2.0   1.0   0.0
  1.0  -2.0   1.0
  0.0   1.0  -2.0
  0.0   0.0   1.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  ⋮          
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0
  0.0   0.0   0.0

In [ ]:
0.000077 seconds (239 allocations: 13.203 KiB)